In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import traitlets
from tkinter import Tk, filedialog

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd 

import os
from os import listdir
from os.path import isfile, join

In [2]:
class DataContainer:
    def __init__(self, filePath):
        self.fileName = filePath
        self.readData()
        
    def readData(self):
        self.data = pd.read_csv(self.fileName, sep=',') 
        
        i=0
        names = []
        while i < len(self.data.columns):
            if i%2 == 1:
                names.append('Unnamed: '+str(i))
            i+=1

        names.append('Unnamed: '+str(len(self.data.columns)-1))  

        for k in names:
            self.data = self.data.drop([k], axis=1)

        self.data = self.data.drop(['S-Polarization Baseline', 'P-Polarization Baseline','Zero Correction'], axis=1)
        self.data = self.data.dropna()

        self.data = self.data.drop(self.data.index[0])
        self.wavelenghts = self.data.index

    def select_data(self,wavelenght,pol):
        ang = self.data.loc[wavelenght].index
        self.iza = []
        self.vza = []
        self.amp = []
        i = 0
        for k in ang:
            split = k.split('/')
            if split[0][0] == pol:
                self.vza.append(int(split[1]))
                self.iza.append(int(split[0][1:]))
                self.amp.append(float(self.data.loc[wavelenght][i]))
            i+=1
        return (self.iza,self.vza,self.amp)

In [3]:
def func(File,Wavelenght,Polarization):
    if File == 'Empty':
        x = [1,2,3,4]
        y = [1,2,3,4]
        z = [1,2,3,4]
        
    else:
        dc = DataContainer(File)
        [x,y,z] = dc.select_data(Wavelenght, Polarization)  

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()
    return (x,y,z)

In [4]:
notebook_path = os.path.abspath("bird_v4-Copy1")
n = listdir(os.path.dirname(notebook_path))
csv_files = []
for i in n:
    if i.lower().endswith( '.csv'):
        csv_files.append(i)

In [5]:
d1 = DataContainer(csv_files[0])

data = widgets.Dropdown(options=csv_files)
pol = widgets.RadioButtons(options=['s','p'])
num = widgets.Dropdown(options=d1.wavelenghts)

files = ['Empty']

def update_wavelenght(change):
    d1 = DataContainer(change.new)
    num.options = d1.wavelenghts

data.observe(update_wavelenght,'value')

k=interactive(func,File=data, Wavelenght=num,Polarization=pol)
display(k)

interactive(children=(Dropdown(description='File', options=('071218_1500_20_60i.csv', '071218_1500_20_60i_ADL.…